In [1]:
import geopandas as gpd
import classes.entropycalculator as ec
import gc
import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

from spatialentropy import altieri_entropy, leibovici_entropy
from scipy.stats import entropy

categorisation = pd.read_excel("data/categorisation.xlsx")

gm_list = ['Eindhoven', 'Amsterdam', 'Rotterdam', 'Utrecht', 'Groningen', 'Maastricht', 'Leeuwarden', 'Arnhem', 'Zwolle', "'s-Gravenhage"]
scales = ['wijken']
filters = [0, 1, 2]

In [2]:
def get_categorisation_filtered(categorisation, L0_filter, L1_filter):
    # Filter the categorisation dataframe based on the L0 and L1 categories
    categorisation = categorisation[~categorisation["L0 category"].isin(L0_filter)]
    if L1_filter:
        for key, value in L1_filter.items():
            categorisation = categorisation[
                ~(
                    (categorisation["L0 category"] == key)
                    & (categorisation["L1 category"].isin(value))
                )
            ]
    L0_cats = categorisation["L0 category"].unique()
    L1_cats = categorisation["L1 category"].unique()
    return L0_cats, L1_cats

In [3]:

# amenities_gm = gpd.read_parquet(f"data/gm_amenities/amenities_{gemeente}.parquet")
wijken = gpd.read_parquet(f"data/wijken_stedent.parquet")
for idx, row in tqdm(wijken.iterrows(), total=wijken.shape[0]): 
    gm_name = row["gemeentenaam"]
    amenities = gpd.read_parquet(f"data/gm_amenities/amenities_{gm_name}.parquet")
    amenities = amenities[amenities.within(row["geometry"])]
    for filteri in [0,1,2]:
        
        L0_filter, L1_filter = ec.getfilter(filteri)
        L0_cats, L1_cats = get_categorisation_filtered(categorisation, L0_filter, L1_filter)

        if amenities.empty:
            for cat in L0_cats:
                wijken.at[idx, f"L0_{filteri}_count_{cat}"] = 0
            for cat in L1_cats:
                wijken.at[idx, f"L1_{filteri}_count_{cat}"] = 0
            continue
            
        L0_counts, L1_counts = amenities.value_counts("L0_category"), amenities.value_counts("L1_category")
        
        for cat in L0_cats:
            wijken.at[idx, f"L0_{filteri}_count_{cat}"] = L0_counts.get(cat, 0)
            
        for cat in L1_cats:
            wijken.at[idx, f"L1_{filteri}_count_{cat}"] = L1_counts.get(cat, 0)
            
    del amenities
    gc.collect()
    
wijken.to_parquet(f"data/wijken_stedent_counts.parquet")
            
        
            

  0%|          | 0/3324 [00:00<?, ?it/s]c:\Users\joepl\anaconda3\envs\scriptie_38\lib\site-packages\geopandas\geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\joepl\anaconda3\envs\scriptie_38\lib\site-packages\geopandas\geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\joepl\anaconda3\envs\scriptie_38\lib\site-packages\geopandas\geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is u

KeyboardInterrupt: 